# Example 3: oo-VQE run on methanimine using Qiskit IBM Runtime Service and a noisy simulator.

In this example, we use `qc2` and the `IBM Quantum Runtime Service` to conduct a oo-VQE calculation on methanimine (CH$_{2}$NH). We utilize the `EstimatorV1` primitive as implemented in the `qiskit-ibm-runtime` package, in conjunction with a noisy simulator.

### Import required packages

In [1]:
import numpy as np

# ASE Atoms object
from ase import Atoms

# IBMQ runtime service packages
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    EstimatorV1 as Estimator,
    Options,
    Session
)

# Qiskit-related packages
from qiskit_aer import AerSimulator
from qiskit_algorithms.optimizers import COBYLA, SLSQP

# ignore package import warnings
import warnings
warnings.filterwarnings('ignore')

# qc2 packages
from qc2.data import qc2Data
from qc2.ase import PySCF
from qc2.algorithms.utils import ActiveSpace
from qc2.algorithms.qiskit import oo_VQE, VQE

# calculated qchem energies with sto-3g basis
CH2NH_HF_ENERGY = -92.82304270237677
CH2NH_CASCI_ENERGY = -92.87111994151458
CH2NH_CASSCF_ENERGY = -92.87254983967073

### Initiate IBM runtime service & prepare `Estimator`

In the following steps, we assume that you have already created an __[IBM Quantum account](https://quantum.ibm.com/)__ and saved your accound credentials (via your personal token) on disk; for further info click __[here](https://github.com/Qiskit/qiskit-ibm-runtime/blob/main/README.md)__.

In [2]:
# instantiating `QiskitRuntimeService`
service = QiskitRuntimeService()

# create a noisy simulator with the same specs of a real backend
real_backend = service.backend("ibm_kyoto")
backend = AerSimulator.from_backend(real_backend)

# Use the following line if you want to run on a real quantum backend
# backend = service.least_busy(simulator=False)

options = Options()

# set seed for reproducible runs
options.simulator.seed_simulator = 42

# error supression options
options.optimization_level = 3

# error mitigation options
options.resilience_level = 2  # ZNE

# execution options
options.execution.shots = 6000

# instantiate the Estimator to calculate expectation values
estimator = Estimator(backend=backend, options=options)

### Instantiate `qc2Data` class and run qc2-ASE calculator

In [3]:
# set Atoms object
mol = Atoms(
    'CH2NH',
    positions=[
        [ 0.058,  0.585, 0.000],
        [-0.831,  1.217, 0.000],
        [-0.932, -1.031, 0.000],
        [ 0.058, -0.688, 0.000],
        [ 1.009,  1.115, 0.000],   
    ]
)

# instantiate qc2Data class
qc2data = qc2Data(
    molecule=mol,
    filename='CH2NH.hdf5',
    schema='qcschema'
)

# specify the qchem calculator
qc2data.molecule.calc = PySCF(method='scf.RHF', basis='sto-3g')

# run calculation and save qchem data in the hdf5 file
qc2data.run()

* Reference energy (Hartree): -92.82304270237677
* Saving qchem data in CH2NH.hdf5



### Instantiate `qc2.algorithms.oo_VQE` class

In [4]:
init_circuit_params = [
    1.46194392e-02, 3.18688670e-07, 1.46198850e-02, -4.16826068e-08,
    -2.48379903e-01, 8.69403138e-08, 1.46800730e-07, -4.14487403e-02
]

init_orbital_params = [
    -3.121019529045726e-16, 2.3936560531586324e-16, -2.512908891659351e-15,
    9.63495665121923e-15, -3.322688177851631e-15, 1.6929996259238916e-15,
    -0.042430832969354224, 0.037788537159730795, -0.6402388305542434,
    1.9747778672836942, -0.42241657801780785, 0.6653997390308073,
    1.8049427015233007e-14, -9.909986054999737e-17, 6.984155203809811e-17,
    -5.740096436152435e-16, 2.9001323479774723e-15, -8.641983286294291e-16,
    1.0876576621088035e-15, -0.022177643413708235, -4.7117310163774974e-17,
    -1.1300943909203134e-05, 4.06199740895308e-06, -3.482280484373978e-05,
    -0.0005134543053396985, -5.2073631321601226e-05, -0.00010725841504334843,
    3.713374803648559e-16, 8.895232986125907e-05, 2.3996887484890763e-16,
    4.100235652869602e-06, -2.886209500963695e-05, 0.001235361499165538,
    0.0004530121939473705, 9.649997855549563e-05, -0.00021739150881453442,
    -3.0049276154810977e-16, 0.00041820453123261165, -8.444748735732234e-16,
    1.330143080213958e-05, 9.479772806843338e-06, -0.0004383044737212205,
    0.000336222549677801, 0.0006018865000642227, -0.0013993665930059293,
    1.1108355579962116e-17, 0.0009727324063734195, -1.7282621334956625e-17,
    7.521744312574251e-06, 6.085143278202279e-05, -0.001335223060730951,
    0.0006296885084617651, -0.0018965057185801933, -0.0023584488500398293,
    -7.391431921310791e-16, 0.0011667046958250059, -1.4475093325379303e-15
]

# instantiate oo-VQE class
qc2data.algorithm = oo_VQE(
    active_space=ActiveSpace(
        num_active_electrons=(2, 2),
        num_active_spatial_orbitals=3
    ),
    mapper="jw",
    optimizer=SLSQP(),
    estimator=estimator,
    max_iterations=100,
    init_circuit_params=init_circuit_params,
    init_orbital_params=init_orbital_params
)

### Run oo-VQE within a `Session`

In [5]:
with Session(service=service, backend=backend) as session:
    results = qc2data.algorithm.run()

>>> Optimizing circuit and orbital parameters...
iter = 000, energy = -92.876207149442 Ha
iter = 001, energy = -92.876734375183 Ha
iter = 002, energy = -92.876925783922 Ha
iter = 003, energy = -92.877010716404 Ha
iter = 004, energy = -92.877054187221 Ha
iter = 005, energy = -92.877077144958 Ha
iter = 006, energy = -92.877087800441 Ha
iter = 007, energy = -92.877092102492 Ha
iter = 008, energy = -92.877095354281 Ha
iter = 009, energy = -92.877098120179 Ha
iter = 010, energy = -92.877098529436 Ha
iter = 011, energy = -92.877097940409 Ha
iter = 012, energy = -92.877098939063 Ha
iter = 013, energy = -92.877100242573 Ha
iter = 014, energy = -92.877099509000 Ha
iter = 015, energy = -92.877098362466 Ha
iter = 016, energy = -92.877099155649 Ha
iter = 017, energy = -92.877100190114 Ha
iter = 018, energy = -92.877099341453 Ha
iter = 019, energy = -92.877098524989 Ha
iter = 020, energy = -92.877099633845 Ha
iter = 021, energy = -92.877100445689 Ha
iter = 022, energy = -92.877099198612 Ha
iter = 0

In [7]:
# print optimized circuit parameters
print(f'* Optimized circuit parameters:')
print(results.optimal_circuit_params, "\n")

# orbital parameters
print(f'* Optimized orbital parameters:')
print(results.optimal_orbital_params, "\n")

print(f'* oo-VQE energy (Hartree):')
print(results.optimal_energy, "\n")

print(f'* CASSCF/sto-3g energy (Hartree):')
print(CH2NH_CASSCF_ENERGY, "\n")

* Optimized circuit parameters:
[ 1.46194392e-02  3.18688670e-07  1.46198850e-02 -4.16826068e-08
 -2.48379903e-01  8.69403138e-08  1.46800730e-07 -4.14487403e-02] 

* Optimized orbital parameters:
[-5.648939194371491e-16, 2.285287159083651e-16, -1.2437415338176084e-14, 1.237880041685819e-14, -4.157188439017333e-15, 1.0979762487595165e-14, -0.04897276627456993, 0.02146896632922398, -1.3089836301242603, 1.4308754844897145, -0.3307408958371843, 1.2631149350530646, 4.76634583444305e-16, 1.65688064711046e-16, -8.189732432435754e-17, 5.581599831555647e-15, -6.200341302141554e-15, 1.1768513763881652e-15, -4.9403599176053935e-15, -0.023244452209662852, -3.931653823813387e-15, 2.8934381218294275e-05, -1.9818451943157152e-05, 0.0011205290775414783, -0.001986404712297885, -0.00011363458996496923, -0.0012469580754804193, 3.5083059513840073e-16, -0.002166270317236903, 2.332358927912693e-16, 1.1629361077227735e-05, -2.6334478602676668e-05, 0.001296312944253889, 0.00025007944717095706, 0.000128277057